# NeqSim Process Equipment Utilization Analysis

This notebook demonstrates how to:
1. Set up various process equipment (pipes, separators, scrubbers, valves, pumps, compressors, heaters)
2. Manually configure equipment sizing parameters for utilization analysis
3. List all constraints for each equipment type
4. Understand the utilization calculation formulas

## Utilization Formula

The general utilization formula is:

$$U = \frac{X_{actual}}{X_{design}} \times 100\%$$

Where:
- $X_{actual}$ = Current operating value (flow, velocity, power, etc.)
- $X_{design}$ = Design/rated capacity

For equipment-specific constraints:
- **Separator**: Gas Load Factor $K = v_g \sqrt{\frac{\rho_g}{\rho_l - \rho_g}}$
- **Pipe**: Erosional velocity ratio
- **Compressor**: Power, speed, surge margin
- **Valve**: Cv utilization

## 1. Import Required Libraries

In [ ]:
from neqsim import jneqsim
from neqsim.thermo import fluid
import pandas as pd

# Process Equipment Classes
Stream = jneqsim.process.equipment.stream.Stream
Separator = jneqsim.process.equipment.separator.Separator
ThreePhaseSeparator = jneqsim.process.equipment.separator.ThreePhaseSeparator
GasScrubber = jneqsim.process.equipment.separator.GasScrubber
ThrottlingValve = jneqsim.process.equipment.valve.ThrottlingValve
Pump = jneqsim.process.equipment.pump.Pump
Compressor = jneqsim.process.equipment.compressor.Compressor
Heater = jneqsim.process.equipment.heatexchanger.Heater
Cooler = jneqsim.process.equipment.heatexchanger.Cooler
Pipe = jneqsim.process.equipment.pipeline.PipeBeggsAndBrills
ProcessSystem = jneqsim.process.processmodel.ProcessSystem

print("NeqSim process equipment classes loaded successfully!")

## 2. Create Fluid and Input Stream

We'll create a natural gas fluid with some water and heavier components to demonstrate multi-phase separation.

In [ ]:
# Create a wet natural gas fluid
gas_fluid = fluid('srk')
gas_fluid.addComponent('nitrogen', 0.02)
gas_fluid.addComponent('CO2', 0.03)
gas_fluid.addComponent('methane', 0.80)
gas_fluid.addComponent('ethane', 0.08)
gas_fluid.addComponent('propane', 0.04)
gas_fluid.addComponent('n-butane', 0.02)
gas_fluid.addComponent('water', 0.01)

# Set inlet conditions
gas_fluid.setTemperature(50.0, 'C')
gas_fluid.setPressure(100.0, 'bara')
gas_fluid.setTotalFlowRate(100000, 'kg/hr')  # 100 tonnes/hr

# Create inlet stream
inlet_stream = Stream("Wellhead-Feed", gas_fluid)

print(f"Inlet Stream: {inlet_stream.getName()}")
print(f"  Temperature: {gas_fluid.getTemperature('C'):.1f} °C")
print(f"  Pressure: {gas_fluid.getPressure('bara'):.1f} bara")
print(f"  Flow Rate: {gas_fluid.getTotalNumberOfMoles():.1f} moles/s")

## 3. Configure Separator with Sizing Parameters

### Separator Utilization Formulas

**Gas Load Factor (Souders-Brown K-value):**
$$K = v_g \sqrt{\frac{\rho_g}{\rho_l - \rho_g}}$$

**Utilization:**
$$U_{gasLoad} = \frac{K_{actual}}{K_{design}} \times 100\%$$

**Other Separator Constraints:**
- Inlet momentum: $\rho v^2$ [Pa] - typically < 16,000 Pa
- Oil retention time - typically > 3 minutes
- Water retention time - typically > 5 minutes
- Droplet cut size - minimum droplet captured [µm]

In [ ]:
# Create separator with sizing parameters
separator = Separator("HP-Separator", inlet_stream)

# =============================================
# MANUALLY SET DESIGN/SIZING PARAMETERS
# =============================================

# Physical dimensions
separator.setInternalDiameter(2.5)      # 2.5 meters internal diameter
separator.setSeparatorLength(8.0)       # 8 meters length

# Design capacity parameters
separator.setDesignGasLoadFactor(0.11)  # Design K-value (m/s) - Souders-Brown
separator.setDesignLiquidLevelFraction(0.5)  # 50% liquid level

# Enable capacity analysis
separator.setCapacityAnalysisEnabled(True)

print("Separator Design Parameters:")
print(f"  Internal Diameter: {separator.getInternalDiameter():.2f} m")
print(f"  Length: {separator.getSeparatorLength():.2f} m")
print(f"  Design Gas Load Factor: {separator.getDesignGasLoadFactor():.4f} m/s")

## 4. Configure Gas Scrubber

Scrubbers are designed for gas-liquid separation with high gas load factors. Similar constraints to separators but optimized for gas handling.

In [ ]:
# Create gas scrubber from separator gas outlet
scrubber = GasScrubber("Gas-Scrubber", separator.getGasOutStream())

# Set scrubber sizing
scrubber.setInternalDiameter(1.8)       # 1.8 meters diameter
scrubber.setSeparatorLength(5.0)        # 5 meters length
scrubber.setDesignGasLoadFactor(0.15)   # Higher K for scrubbers
scrubber.setCapacityAnalysisEnabled(True)

print("Scrubber Design Parameters:")
print(f"  Internal Diameter: {scrubber.getInternalDiameter():.2f} m")
print(f"  Length: {scrubber.getSeparatorLength():.2f} m")
print(f"  Design Gas Load Factor: {scrubber.getDesignGasLoadFactor():.4f} m/s")

## 5. Configure Throttling Valve

### Valve Utilization Formula

**Flow Coefficient (Cv):**
$$C_v = \frac{Q \cdot N_1}{\sqrt{\frac{\Delta P \cdot G_f}{F_p^2}}}$$

**Cv Utilization:**
$$U_{Cv} = \frac{C_{v,required}}{C_{v,installed}} \times 100\%$$

**Valve Opening:**
- Optimal operating range: 20-80% open
- Full open (100%) indicates undersized valve

In [ ]:
# Create throttling valve for pressure letdown
valve = ThrottlingValve("LP-Valve", scrubber.getGasOutStream())

# Set valve parameters
valve.setOutletPressure(30.0)  # Reduce to 30 bara

# Set design Cv (flow coefficient) - SIZING PARAMETER
valve.setCv(500.0)  # Installed Cv capacity
valve.setCapacityAnalysisEnabled(True)

print("Valve Design Parameters:")
print(f"  Outlet Pressure: {valve.getOutletPressure():.1f} bara")
print(f"  Installed Cv: {valve.getCv():.1f}")

## 6. Configure Pipe

### Pipe Utilization Formulas

**Erosional Velocity:**
$$v_{erosional} = \frac{C}{\sqrt{\rho_m}}$$

where C ≈ 100-150 for clean gas, $\rho_m$ is mixture density (kg/m³)

**Velocity Utilization:**
$$U_{velocity} = \frac{v_{actual}}{v_{erosional}} \times 100\%$$

**Pressure Drop:** Should typically be < 5-10% of inlet pressure

In [ ]:
# Create pipe section
pipe = Pipe("Export-Pipe", valve.getOutletStream())

# Set pipe sizing parameters
pipe.setDiameter(0.3)           # 12 inch (0.3m) diameter
pipe.setLength(1000.0)          # 1000 meters length
pipe.setPipeWallRoughness(0.00005)  # Roughness in meters
pipe.setNumberOfIncrements(20)  # Calculation segments

# Optionally set elevation change
pipe.setElevation(0.0)          # Horizontal pipe

print("Pipe Design Parameters:")
print(f"  Diameter: {pipe.getDiameter():.3f} m ({pipe.getDiameter()*39.37:.1f} inches)")
print(f"  Length: {pipe.getLength():.1f} m")
print(f"  Wall Roughness: {pipe.getPipeWallRoughness():.6f} m")

## 7. Configure Compressor

### Compressor Utilization Formulas

**Power Utilization:**
$$U_{power} = \frac{P_{actual}}{P_{design}} \times 100\%$$

**Speed Utilization:**
$$U_{speed} = \frac{N_{actual}}{N_{max}} \times 100\%$$

**Surge Margin:** Distance from surge line (should be > 10%)
$$Surge\,Margin = \frac{Q_{actual} - Q_{surge}}{Q_{actual}} \times 100\%$$

In [ ]:
# Create compressor for gas recompression
compressor = Compressor("Export-Compressor", pipe.getOutletStream())

# Set compressor operating parameters
compressor.setOutletPressure(80.0)  # Recompress to 80 bara
compressor.setPolytropicEfficiency(0.78)  # 78% polytropic efficiency
compressor.setUsePolytropicCalc(True)

# Set compressor design/sizing parameters for utilization analysis
compressor.setMaximumSpeed(10000)  # Maximum RPM
compressor.setSpeed(8500)  # Current operating speed
compressor.setDesignPower(5000)  # Design power in kW

compressor.setCapacityAnalysisEnabled(True)

print("Compressor Design Parameters:")
print(f"  Outlet Pressure: {compressor.getOutletPressure():.1f} bara")
print(f"  Polytropic Efficiency: {compressor.getPolytropicEfficiency()*100:.1f}%")
print(f"  Maximum Speed: {compressor.getMaximumSpeed():.0f} RPM")
print(f"  Current Speed: {compressor.getSpeed():.0f} RPM")

## 8. Configure Pump

### Pump Utilization Formulas

**Flow Utilization:**
$$U_{flow} = \frac{Q_{actual}}{Q_{design}} \times 100\%$$

**Power Utilization:**
$$U_{power} = \frac{P_{actual}}{P_{rated}} \times 100\%$$

**NPSH Margin:**
$$NPSH\,Margin = NPSH_{available} - NPSH_{required}$$

Must be > 0 to prevent cavitation

In [ ]:
# Create pump for liquid from separator
pump = Pump("Condensate-Pump", separator.getLiquidOutStream())

# Set pump operating parameters
pump.setOutletPressure(150.0)  # Pump to 150 bara

# Set pump design parameters for utilization
pump.setMaximumFlowCapacity(500.0)  # Maximum flow in m³/hr
pump.setDesignPower(200.0)  # Design power in kW

pump.setCapacityAnalysisEnabled(True)

print("Pump Design Parameters:")
print(f"  Outlet Pressure: {pump.getOutletPressure():.1f} bara")
print(f"  Maximum Flow Capacity: {pump.getMaximumFlowCapacity():.1f} m³/hr")

## 9. Configure Heater/Cooler

### Heat Exchanger Utilization Formulas

**Duty Utilization:**
$$U_{duty} = \frac{Q_{actual}}{Q_{design}} \times 100\%$$

**Approach Temperature:**
$$\Delta T_{approach} = T_{hot,out} - T_{cold,in}$$

Minimum approach typically 5-10°C for economic design

In [ ]:
# Create heater for gas heating before export
heater = Heater("Export-Heater", compressor.getOutletStream())

# Set heater parameters
heater.setOutTemperature(60.0 + 273.15)  # Heat to 60°C (input in Kelvin)

# Set design duty for utilization analysis
heater.setDesignDuty(2000.0)  # Design duty in kW

heater.setCapacityAnalysisEnabled(True)

# Also create a cooler example
cooler = Cooler("Aftercooler", heater.getOutletStream())
cooler.setOutTemperature(30.0 + 273.15)  # Cool to 30°C
cooler.setDesignDuty(1500.0)  # Design duty in kW
cooler.setCapacityAnalysisEnabled(True)

print("Heater Design Parameters:")
print(f"  Outlet Temperature: 60.0 °C")
print(f"  Design Duty: {heater.getDesignDuty():.1f} kW")
print("\nCooler Design Parameters:")
print(f"  Outlet Temperature: 30.0 °C")
print(f"  Design Duty: {cooler.getDesignDuty():.1f} kW")

## 10. Build and Run Process System

In [ ]:
# Create process system and add all equipment
process = ProcessSystem()

# Add equipment in process order
process.add(inlet_stream)
process.add(separator)
process.add(scrubber)
process.add(valve)
process.add(pipe)
process.add(compressor)
process.add(heater)
process.add(cooler)
process.add(pump)

# Run the simulation
print("Running process simulation...")
process.run()
print("Simulation complete!")

# Show number of equipment units
print(f"\nTotal equipment units: {process.getUnitOperations().size()}")

## 11. List All Equipment and Their Constraints

Now we'll iterate through all equipment and display their constraints and utilization values.

In [ ]:
# Function to list all equipment with their constraints
def list_equipment_constraints(process_system):
    """List all equipment and their capacity constraints"""
    
    print("=" * 80)
    print("PROCESS EQUIPMENT UTILIZATION SUMMARY")
    print("=" * 80)
    
    ops = process_system.getUnitOperations()
    
    for i in range(ops.size()):
        unit = ops.get(i)
        unit_type = str(type(unit)).split('.')[-1].replace("'>", "")
        
        print(f"\n{'─' * 80}")
        print(f"📦 {i+1}. {unit.getName()} ({unit_type})")
        print(f"{'─' * 80}")
        print(f"   Specification: {unit.getSpecification()}")
        
        # Check for capacity constraints
        if hasattr(unit, 'getCapacityConstraints'):
            constraints = unit.getCapacityConstraints()
            if constraints and constraints.size() > 0:
                print(f"   Max Utilization: {unit.getMaxUtilizationPercent():.1f}%")
                print(f"   Available Margin: {unit.getAvailableMarginPercent():.1f}%")
                print(f"   Is Capacity Exceeded: {unit.isCapacityExceeded()}")
                print()
                print("   CONSTRAINTS:")
                print("   " + "-" * 70)
                
                for key in constraints.keySet():
                    c = constraints.get(key)
                    status = "⚠️ VIOLATED" if c.isViolated() else "✓ OK"
                    print(f"   • {c.getName():25s} = {c.getCurrentValue():12.4f} {c.getUnit():10s} "
                          f"({c.getUtilizationPercent():6.1f}% of design) [{status}]")

# Run the listing
list_equipment_constraints(process)

## 12. Detailed Separator Analysis

The separator has multiple constraints that are checked simultaneously. Let's examine them in detail.

In [ ]:
# Detailed separator constraint analysis
print("=" * 60)
print("SEPARATOR DETAILED ANALYSIS")
print("=" * 60)

print(f"\n📊 Design Parameters:")
print(f"   Internal Diameter: {separator.getInternalDiameter():.2f} m")
print(f"   Length: {separator.getSeparatorLength():.2f} m")
print(f"   Design Gas Load Factor (K): {separator.getDesignGasLoadFactor():.4f} m/s")

print(f"\n📈 Operating Values:")
print(f"   Actual Gas Load Factor: {separator.getGasLoadFactor():.4f} m/s")
print(f"   Gas Superficial Velocity: {separator.getGasSuperficialVelocity():.4f} m/s")

print(f"\n⚙️ Utilization Metrics:")
print(f"   Capacity Utilization: {separator.getCapacityUtilization():.1f}%")
print(f"   Max Utilization: {separator.getMaxUtilizationPercent():.1f}%")
print(f"   Available Margin: {separator.getAvailableMarginPercent():.1f}%")

print(f"\n✅ Constraint Status:")
print(f"   Is Within Operating Envelope: {separator.isWithinOperatingEnvelope()}")
print(f"   Is Capacity Exceeded: {separator.isCapacityExceeded()}")
print(f"   Is Hard Limit Exceeded: {separator.isHardLimitExceeded()}")
print(f"   Is K-Value Within Limit: {separator.isKValueWithinLimit()}")
print(f"   Is Inlet Momentum Within Limit: {separator.isInletMomentumWithinLimit()}")

# Get constraint summary text
print(f"\n📋 Constraint Summary:")
print(separator.getConstraintSummary())

## 13. Create Utilization Summary DataFrame

Create a summary table showing all equipment and their key utilization metrics.

In [ ]:
# Create summary DataFrame
summary_data = []

ops = process.getUnitOperations()
for i in range(ops.size()):
    unit = ops.get(i)
    unit_type = str(type(unit)).split('.')[-1].replace("'>", "")
    
    row = {
        'Equipment': unit.getName(),
        'Type': unit_type,
        'Specification': unit.getSpecification(),
    }
    
    # Add utilization data if available
    if hasattr(unit, 'getMaxUtilizationPercent'):
        row['Max Utilization (%)'] = round(unit.getMaxUtilizationPercent(), 1)
        row['Available Margin (%)'] = round(unit.getAvailableMarginPercent(), 1)
        row['Capacity Exceeded'] = unit.isCapacityExceeded()
        
        # Get bottleneck constraint
        if hasattr(unit, 'getBottleneckConstraint'):
            bottleneck = unit.getBottleneckConstraint()
            if bottleneck:
                row['Bottleneck Constraint'] = bottleneck.getName()
            else:
                row['Bottleneck Constraint'] = '-'
    else:
        row['Max Utilization (%)'] = '-'
        row['Available Margin (%)'] = '-'
        row['Capacity Exceeded'] = '-'
        row['Bottleneck Constraint'] = '-'
    
    summary_data.append(row)

# Create DataFrame
summary_df = pd.DataFrame(summary_data)
summary_df

## 14. Setting Design Capacities for Utilization Analysis

### Summary of Key Sizing Methods by Equipment Type

| Equipment | Method | Description |
|-----------|--------|-------------|
| **Separator** | `setInternalDiameter(m)` | Vessel diameter |
| | `setSeparatorLength(m)` | Vessel length |
| | `setDesignGasLoadFactor(m/s)` | K-value design capacity |
| **Valve** | `setCv(-)` | Flow coefficient |
| **Compressor** | `setDesignPower(kW)` | Design shaft power |
| | `setMaximumSpeed(RPM)` | Maximum rotational speed |
| **Pump** | `setMaximumFlowCapacity(m³/hr)` | Design flow capacity |
| | `setDesignPower(kW)` | Design power |
| **Heater/Cooler** | `setDesignDuty(kW)` | Design heat duty |
| **Pipe** | `setDiameter(m)` | Pipe internal diameter |

In [ ]:
# Example: Modify design capacities and re-analyze

# Reduce separator size to increase utilization
print("Before resizing:")
print(f"  Separator Gas Load Factor: {separator.getGasLoadFactor():.4f} m/s")
print(f"  Design Gas Load Factor: {separator.getDesignGasLoadFactor():.4f} m/s")
print(f"  Capacity Utilization: {separator.getCapacityUtilization():.1f}%")

# Change design parameters
separator.setInternalDiameter(1.5)  # Smaller vessel
separator.setDesignGasLoadFactor(0.08)  # Lower design K-value

# Re-run to recalculate
process.run()

print("\nAfter resizing (smaller vessel):")
print(f"  Separator Gas Load Factor: {separator.getGasLoadFactor():.4f} m/s")
print(f"  Design Gas Load Factor: {separator.getDesignGasLoadFactor():.4f} m/s")
print(f"  Capacity Utilization: {separator.getCapacityUtilization():.1f}%")
print(f"  Is Capacity Exceeded: {separator.isCapacityExceeded()}")

## 15. Summary of Utilization Formulas

### General Utilization Formula
$$Utilization = \frac{Actual\:Operating\:Value}{Design\:Capacity} \times 100\%$$

### Equipment-Specific Formulas

#### Separator
$$U_{gasLoad} = \frac{K_{actual}}{K_{design}} \times 100\%$$

Where: $K = v_g \sqrt{\frac{\rho_g}{\rho_l - \rho_g}}$ (Souders-Brown equation)

#### Valve
$$U_{Cv} = \frac{C_{v,required}}{C_{v,installed}} \times 100\%$$

#### Compressor
$$U_{power} = \frac{Power_{actual}}{Power_{design}} \times 100\%$$
$$U_{speed} = \frac{N_{actual}}{N_{max}} \times 100\%$$

#### Pump
$$U_{flow} = \frac{Q_{actual}}{Q_{design}} \times 100\%$$

#### Heat Exchanger
$$U_{duty} = \frac{Q_{actual}}{Q_{design}} \times 100\%$$

#### Pipe
$$U_{velocity} = \frac{v_{actual}}{v_{erosional}} \times 100\%$$

Where: $v_{erosional} = \frac{C}{\sqrt{\rho_m}}$ (C ≈ 100-150)

### Constraint Status
- **OK**: Utilization < 100%
- **VIOLATED**: Utilization > 100%
- **NEAR LIMIT**: Utilization > warning threshold (typically 90%)

## 16. Available Methods Reference

Let's list all useful methods for constraint analysis on a separator.

In [ ]:
# List useful constraint/utilization methods on separator
print("=" * 60)
print("SEPARATOR METHODS FOR UTILIZATION ANALYSIS")
print("=" * 60)

separator_methods = [
    # Constraint methods
    ("getCapacityConstraints()", "Get all constraints as a Map"),
    ("getBottleneckConstraint()", "Get the limiting constraint"),
    ("getConstraintSummary()", "Get text summary of constraints"),
    ("getUtilizationSummary()", "Get Map of utilization percentages"),
    
    # Utilization checks
    ("getMaxUtilizationPercent()", "Maximum utilization across all constraints"),
    ("getAvailableMarginPercent()", "Remaining headroom percentage"),
    ("getCapacityUtilization()", "Overall capacity utilization"),
    ("isCapacityExceeded()", "True if any constraint > 100%"),
    ("isHardLimitExceeded()", "True if hard limits violated"),
    ("isWithinOperatingEnvelope()", "True if all constraints OK"),
    ("isNearCapacityLimit()", "True if near warning threshold"),
    
    # Specific separator checks
    ("isKValueWithinLimit()", "Check K-value constraint"),
    ("isInletMomentumWithinLimit()", "Check inlet momentum"),
    ("isOilRetentionTimeAboveMinimum()", "Check oil retention time"),
    ("isWaterRetentionTimeAboveMinimum()", "Check water retention time"),
    
    # Calculated values
    ("getGasLoadFactor()", "Current K-value (m/s)"),
    ("getDesignGasLoadFactor()", "Design K-value (m/s)"),
    ("calcInletMomentumFlux()", "Calculate inlet momentum (Pa)"),
    ("calcOilRetentionTime()", "Calculate oil retention (min)"),
    ("calcWaterRetentionTime()", "Calculate water retention (min)"),
]

for method, description in separator_methods:
    print(f"  {method:40s} - {description}")

print("\n" + "=" * 60)
print("CONSTRAINT OBJECT METHODS")
print("=" * 60)
constraint_methods = [
    ("getName()", "Constraint name"),
    ("getCurrentValue()", "Current operating value"),
    ("getDesignValue()", "Design/rated value"),
    ("getMaxValue()", "Maximum allowable value"),
    ("getUnit()", "Unit of measurement"),
    ("getUtilizationPercent()", "Utilization as percentage"),
    ("isViolated()", "True if > 100% utilization"),
    ("isEnabled()", "True if constraint is active"),
]

for method, description in constraint_methods:
    print(f"  {method:30s} - {description}")